In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

# from matplotlib import style
import seaborn as sns
%matplotlib inline

import wrangle

In [2]:
df = wrangle.get_curriculum_logs_data()

The df has 847330 rows and 10 columns.


In [3]:
df.head()

,cohortid,start_date,end_date,cohortname,program_id,date,time,path,user_id,ip,program_name,is_staff
0,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:14:47,/,11,97.105.19.61,web_dev,False
1,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:14:51,javascript-i,11,97.105.19.61,web_dev,False
2,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:15:02,javascript-i/functions,11,97.105.19.61,web_dev,False
3,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,11:46:29,appendix/further-reading/javascript/hoisting.html,11,97.105.19.61,web_dev,False
4,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,11:46:39,appendix,11,97.105.19.61,web_dev,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   cohortid      847330 non-null  int64         
 1   start_date    847330 non-null  datetime64[ns]
 2   end_date      847330 non-null  datetime64[ns]
 3   cohortname    847330 non-null  object        
 4   program_id    847330 non-null  int64         
 5   date          847330 non-null  datetime64[ns]
 6   time          847330 non-null  object        
 7   path          847329 non-null  object        
 8   user_id       847330 non-null  int64         
 9   ip            847330 non-null  object        
 10  program_name  847330 non-null  object        
 11  is_staff      847330 non-null  bool          
dtypes: bool(1), datetime64[ns](3), int64(3), object(5)
memory usage: 78.4+ MB


### Q3: Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [5]:
# Active access
df_active = df[(df.start_date <= df.date) & (df.date <= df.end_date)]

In [6]:
df_active.shape

(647496, 12)

In [28]:
# Curriculum access by counting the paths per user_id
df_active.groupby('user_id').path.count().describe()

count     731.000000
mean      885.766074
std       647.254908
min         1.000000
25%       391.500000
50%       795.000000
75%      1235.500000
max      4786.000000
Name: path, dtype: float64

In [8]:
# first quartile
q1 = np.quantile(df_active.groupby('user_id').path.count(), .25)

In [9]:
# find user id's in the first quartile of access count
inq1 = pd.DataFrame(df_active.groupby('user_id').path.count() < q1)
inq1.rename(columns = {'path':'in_q1'}, inplace=True)

In [10]:
# add column if the user_id is in the first quartile
df_active = pd.merge(df_active, inq1, how = 'right', on='user_id')

In [15]:
# dataframe of students who hardly accessed curriculum while active
df_swhacwa = df_active[df_active.in_q1 == True]

In [16]:
# Number of students who hardly accessed curriculum while active.
len(df_swhacwa.user_id.unique())

183

In [17]:
df_swhacwa.groupby('user_id').path.count().mean()

163.02185792349727

In [52]:
len(df_swhacwa[df_swhacwa.program_name == 'data_science'].user_id.unique())

21

In [53]:
len(df_swhacwa[df_swhacwa.program_name == 'web_dev'].user_id.unique())

163

In [24]:
df_swhacwa.path.str[:8].value_counts()

javascri    7861
html-css    5976
/           1813
fundamen    1689
spring/f    1398
            ... 
data-sci       1
joins          1
css            1
slides/e       1
content/       1
Name: path, Length: 177, dtype: int64

#### Students who hardly accessed the curriculum while active info
- 183 students accessed the curriculum < first quartile (391 times)
- 163 of these students are from web_dev programs
- 21 are from data science programs
- 1 student is in both programs
- Average access count of these students is 163 times
- ~ 30k total access count